In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pickle
import sys

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

sys.path.insert(0,'../charles/helpers')
import helper
import joblib

In [2]:
data = pd.read_csv('../communal/Ames_Housing_Price_Data_cleaned_2.csv', header = [0])

pd.set_option("display.max_columns", None)

data.drop(['PID', 'lot_bucket', 'mean_LotFrontage', 'Prop_Addr', 'GarageYrBlt', 'lat', 'long'], axis = 1, inplace = True)

data.drop(['YearRemodAdd', 'Exterior2nd', 'BsmtFinSF1', 'BsmtFinSF2',
           'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 
           'TotRmsAbvGrd', 'GarageCars'], axis = 1, inplace = True)
data['Total_Porch'] = data['OpenPorchSF'] + data['EnclosedPorch'] + \
                      data['3SsnPorch'] + data['ScreenPorch']
data.drop(['OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch'],
           axis = 1, inplace = True)

In [3]:
y = data['SalePrice'].apply(lambda x: np.log(x))
x = data.drop(['SalePrice'], axis = 1)
x = pd.get_dummies(x, drop_first = True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(x,y)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

params = {'alpha': np.linspace(1e-7,1e-3,1000)}

lasso_gs = GridSearchCV(Lasso(max_iter = 500000),param_grid=params, cv = 5)

In [5]:
lasso_gs = helper.runGridSearch(lasso_gs, X_train, X_test, y_train, y_test, 'lasso_all', 'all features')

joblib imported
change made


In [6]:
lasso_gs.best_estimator_.alpha

0.0002763486486486487

In [7]:
lasso_gs.score(X_train, y_train)

0.9299682502474976

In [8]:
lasso_gs.score(X_test, y_test)

0.9187567546261017

In [ ]:
lasso_gs_df = pd.DataFrame(X_train, columns = x.columns)

In [9]:
coefs = lasso_gs.best_estimator_.coef_
coef_list = list(zip(coefs, x.columns))
sorted(coef_list, key = lambda x: abs(x[0]), reverse = True)

[(1.180384602161, 'GrLivArea'),
 (0.4847462166810315, 'OverallQual'),
 (0.3312488631315472, 'OverallCond'),
 (0.30737954062840084, 'TotalBsmtSF'),
 (-0.29657801637518394, 'Condition2_PosN'),
 (0.21883367020809033, 'YearBuilt'),
 (0.20014337731954884, 'GarageArea'),
 (-0.19666548747302695, 'MSZoning_C (all)'),
 (0.12139130445563352, 'LotArea'),
 (0.10562448289295315, 'Neighborhood_Crawfor'),
 (-0.09952009128877051, 'Neighborhood_MeadowV'),
 (0.09867607043850014, 'Fireplaces'),
 (0.09598389216017113, 'BsmtFullBath'),
 (0.08727640152733482, 'Total_Porch'),
 (0.08517119797043032, 'Neighborhood_Somerst'),
 (-0.08092714067756943, 'BldgType_Twnhs'),
 (0.07896175112418229, 'SaleCondition_Partial'),
 (-0.07647051597769913, 'ExterCond_Fa'),
 (0.0744969710772724, 'Exterior1st_BrkFace'),
 (0.06957188464007424, 'CentralAir_Y'),
 (0.06836112616578296, 'Neighborhood_StoneBr'),
 (-0.0631356366353868, 'BsmtExposure_None'),
 (-0.06071534634555036, 'BsmtQual_Fa'),
 (-0.05996552564907705, 'BldgType_Duplex

In [12]:
x_train_df = pd.DataFrame(X_train, columns = x.columns)

In [13]:
x_test_df = pd.DataFrame(X_test, columns = x.columns)

In [15]:
def sub_lists (lst):
    lists = [[]]
    for i in range(len(lst) + 1):
        for j in range(i):
            lists.append(lst[j:i])
    return lists

In [30]:
col_combinations = sub_lists(list(x_test_df.columns))[1:]

In [33]:
score_dict = {}
for c in col_combinations:
    score = lasso_gs.score(x_test_df[c], y_test)
    score_dict[c] = score

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 239 is different from 1)